In [1]:
import pandas as pd 
import numpy as np 

In [3]:
#importamos los datasets previamente transformados
#Primero ponemos la ruta
ruta_games = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\games.parquet"
ruta_items = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\users_items.parquet"
ruta_reviews = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\df_review_final.parquet"
#luego los extraemos
games = pd.read_parquet(ruta_games)
items = pd.read_parquet(ruta_items)
reviews = pd.read_parquet(ruta_reviews)

In [8]:
# Se extraen las columnas que se necesitan
items_col = items[["user_id", "items_count", "item_id"]]
reviews_col = reviews[["User_Id", "Item_Id", "Recommend"]]
games_col = games[["Id", "Price"]]

In [35]:
# Se normalizan los nombres de las columnas
items_col.columns = items_col.columns.str.replace('_', ' ').str.title().str.replace(' ', '_')
# Se cambia el nombre de las columnas
games_col.rename(columns={'Id':'Item_Id'}, inplace=True)

C:\Users\Cristian\AppData\Local\Temp\ipykernel_8604\1839938983.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_col.rename(columns={'Id':'Item_Id'}, inplace=True)


In [41]:
# Eliminamos nulos
items_col = items_col.dropna(subset=['Item_Id'])
reviews_col = reviews_col.dropna(subset=['Item_Id'])
games_col = games_col.dropna(subset=['Item_Id'])

# Convertimos columnas al mismo tipo
items_col["Item_Id"] = items_col["Item_Id"].astype("int64")
reviews_col["Item_Id"] = reviews_col["Item_Id"].astype("int64")
games_col["Item_Id"] = games_col["Item_Id"].astype("int64") 
# Ahora unimos los datasets

dataset_1 = pd.merge(items_col, reviews_col, on=["User_Id", "Item_Id"], how = "inner")
funcion_2 = pd.merge(dataset_1, games_col, on= "Item_Id", how= "inner")
funcion_2.head(5)

,User_Id,Items_Count,Item_Id,Recommend,Price
0,76561197970982479,277,22200,True,9.99
1,76561197970982479,277,1250,True,19.99
2,js41637,888,227300,True,19.99
3,js41637,888,239030,True,9.99
4,evcentric,137,107200,True,9.99


In [52]:
#def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
#Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

def userdata(User_id : str):
    '''
    Esta función devuelve información sobre un usuario según su 'user_id'.

    Args:
        user_id (str): Identificador único del usuario.

    Returns:
        dict: Un diccionario que contiene información sobre el usuario.
            - 'cantidad_dinero' (int): Cantidad de dinero gastado por el usuario.
            - 'porcentaje_recomendacion' (float): Porcentaje de recomendaciones realizadas por el usuario.
            - 'total_items' (int): Cantidad de items que tiene el usuario.
    '''
     # Obtenemos solamente la informacion relacionada con el usuario
    user_data = funcion_2[funcion_2['User_Id'] == User_id]
    
    if user_data.empty:
        return f"No se encontraron datos para el usuario {User_id}"
    
    # Cantidad de dinero gastado
    dinero_gastado = user_data['Price'].sum().round(2)

    # Porcentaje de recomendación
    total_reviews = len(user_data)
    recomendaciones = user_data[user_data['Recommend'] == True]
    porcentaje_recomendacion = len(recomendaciones) / total_reviews * 100 if total_reviews > 0 else 0

    # Cantidad de items 
    cantidad_items = user_data['Items_Count'].iloc[0]  

    return {
        "Usuario": User_id,
        "Dinero gastado": f"{dinero_gastado} USD",
        "% de recomendación": f"{porcentaje_recomendacion:.2f}%",
        "Cantidad de items": cantidad_items
    }


In [55]:
print(userdata('76561197970982479'))

{'Usuario': '76561197970982479', 'Dinero gastado': '29.98 USD', '% de recomendación': '100.00%', 'Cantidad de items': 277}


In [56]:
funcion_2.to_parquet("Dataset_funcion_2.parquet", engine = "pyarrow") #Exportamos el dataset